In [1]:
!pip install requests

In [3]:
!pip install stix2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 2.4 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=d95fe0eec952999b0f89f3b8d31c373c361d4509a61f959397f8dcc403a15585
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antlr4-python3-runtime


In [5]:
!apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [6]:
!git clone https://github.com/mitre/cti.git

Cloning into 'cti'...
remote: Enumerating objects: 474670, done.
remote: Counting objects: 100% (70172/70172), done.
remote: Compressing objects: 100% (2218/2218), done.
remote: Total 474670 (delta 67963), reused 70137 (delta 67954), pack-reused 404498 (from 1)
Receiving objects: 100% (474670/474670), 227.06 MiB | 19.95 MiB/s, done.
Resolving deltas: 100% (461480/461480), done.
Updating files: 100% (29393/29393), done.


In [25]:
from stix2 import FileSystemSource, Filter

In [1]:
import requests

In [32]:
parts = [
    'a', # Application
    'o', # Operating System
    'h', # Hardware
    'a|o', # Application or Operating System
    'a|h', # Application or Hardware
    'o|h', # Operating System or Hardware
    'a|o|h' # Application, Operating System, or Hardware
         ]

In [26]:
def get_related_cwes_by_capec_id(capec_id):
    fs = FileSystemSource('./cti/capec/2.1')

    filt = [
        Filter('type', '=', 'attack-pattern'),
        Filter('external_references.external_id', '=', 'CAPEC-' + capec_id),
        Filter('external_references.source_name', '=', 'capec'),
    ]

    attack_pattern = fs.query(filt)[0]

    cwe_ids = [ref['external_id'] for ref in attack_pattern['external_references'] if ref['source_name'] == 'cwe']

    return cwe_ids

In [53]:
def get_related_cwes_and_attributes_by_capec_id(capec_id):
    fs = FileSystemSource('./cti/capec/2.1')

    # Define the filter to query for the specific CAPEC attack pattern
    filt = [
        Filter('type', '=', 'attack-pattern'),
        Filter('external_references.external_id', '=', 'CAPEC-' + capec_id),
        Filter('external_references.source_name', '=', 'capec'),
    ]

    # Query the attack pattern
    attack_pattern = fs.query(filt)[0]

    # Retrieve CWE IDs
    cwe_ids = [ref['external_id'] for ref in attack_pattern['external_references'] if ref['source_name'] == 'cwe']

    # Retrieve additional attributes
    name = attack_pattern.get('name', '')
    prerequisites = attack_pattern.get('x_capec_prerequisites', [])
    resources_required = attack_pattern.get('x_capec_resources_required', [])
    skills_required = attack_pattern.get('x_capec_skills_required', {})
    likelihood_of_attack = attack_pattern.get('x_capec_likelihood_of_attack', '')
    consequences = attack_pattern.get('x_capec_consequences', {})

    # Return a dictionary with all relevant information
    return {
        'cwe_ids': cwe_ids,
        'name': name,
        'prerequisites': prerequisites,
        'resources_required': resources_required,
        'skills_required': skills_required,
        'likelihood_of_attack': likelihood_of_attack,
        'consequences': consequences
    }

In [50]:
def create_capec_dictionary(capec_ids):
    capecs = {}

    for capec_id in capec_ids:
        try:
            capecs[capec_id] = get_related_cwes_and_attributes_by_capec_id(capec_id)
        except Exception as e:
            print(f"Error retrieving data for CAPEC-{capec_id}: {e}")

    return capecs

In [54]:
# Example usage
capec_ids = []
for i in range(3):
  capec_ids.append(input("Enter CAPEC ID: "))

capec_dictionary = create_capec_dictionary(capec_ids)

# Print the results
for capec_id, info in capec_dictionary.items():
    print(f"CAPEC-{capec_id}:")
    print(f"  Name: {info['name']}")
    print(f"  CWE IDs: {info['cwe_ids']}")
    print(f"  Prerequisites: {info['prerequisites']}")
    print(f"  Resources Required: {info['resources_required']}")
    print(f"  Skills Required: {info['skills_required']}")
    print(f"  Likelihood of Attack: {info['likelihood_of_attack']}")
    print(f"  Consequences: {info['consequences']}\n")

Enter CAPEC ID: 112
Enter CAPEC ID: 114
Enter CAPEC ID: 20
CAPEC-112:
  Name: Brute Force
  CWE IDs: ['CWE-330', 'CWE-326', 'CWE-521']
  Prerequisites: ['The attacker must be able to determine when they have successfully guessed the secret. As such, one-time pads are immune to this type of attack since there is no way to determine when a guess is correct.']
  Resources Required: ['None: No specialized resources are required to execute this type of attack. Ultimately, the speed with which an attacker discovers a secret is directly proportional to the computational resources the attacker has at their disposal. This attack method is resource expensive: having large amounts of computational power do not guarantee timely success, but having only minimal resources makes the problem intractable against all but the weakest secret selection procedures.']
  Skills Required: {'Low': 'The attack simply requires basic scripting ability to automate the exploration of the search space. More sophistic

In [60]:
def fetch_cves_with_cwe(cwe_id):
    # NVD API endpoint for CVE data
    nvd_url = "https://services.nvd.nist.gov/rest/json/cves/2.0"

    # Construct the CPE string for querying
    #cpe_string = f"?cpeName=cpe:2.3:a:{asset_name}:{asset_version}"
    cwe_string = f"?cweId=CWE-{cwe_id}"
    # Query the NVD API for CVEs related to the specified asset
    response = requests.get(f"{nvd_url}{cwe_string}")

    if response.status_code == 200:
        data = response.json()
        #print(data)
        # Initialize a list to store CVE IDs
        tot_res = data['totalResults']
        res_per_page = data['resultsPerPage']
        results_start = data['startIndex']

        if not tot_res:
            print("\tNo CVEs found for this asset.")
            return
        print(f"\tTotal results {tot_res}\n\tTotal result per page {res_per_page}")
        cve_list = []
        vulnerabilities = data['vulnerabilities']

        # Extract CVE IDs from the vulnerabilities section
        for vulnerability in vulnerabilities:
          cve = vulnerability['cve']
          cve_id = cve['id']
          cve_list.append(cve_id)

        # Print the list of CVE IDs
        print("\tList of CVEs:")
        for cve in cve_list:
            print(f"\t\t{cve}")

    else:
        print(f"Failed to fetch data: {response.status_code}")

In [61]:
# Look for CVEs related to the specified CWEs
for capec_id, info in capec_dictionary.items():
  print(f"CAPEC ID: {capec_id}")
  related_cwes = info['cwe_ids']

  for cwe in related_cwes:
    cwe_id = cwe[4:]
    print(f"\tCWE ID: {cwe_id}\n")
    fetch_cves_with_cwe(cwe_id)
    print("\n")

CAPEC ID: 112
	CWE ID: 330

	Total results 294
	Total result per page 294
	List of CVEs:
		CVE-2008-0141
		CVE-2008-0087
		CVE-2008-2020
		CVE-2008-2433
		CVE-2008-3612
		CVE-2008-4905
		CVE-2008-4929
		CVE-2008-5162
		CVE-2009-0255
		CVE-2009-2158
		CVE-2013-6925
		CVE-2015-3963
		CVE-2016-5085
		CVE-2016-5100
		CVE-2015-9019
		CVE-2013-7463
		CVE-2017-6026
		CVE-2017-7901
		CVE-2017-7902
		CVE-2017-7905
		CVE-2017-13077
		CVE-2017-13078
		CVE-2017-13079
		CVE-2017-13080
		CVE-2017-13081
		CVE-2017-13082
		CVE-2017-13084
		CVE-2017-13086
		CVE-2017-13087
		CVE-2017-13088
		CVE-2017-1000246
		CVE-2017-12361
		CVE-2017-10874
		CVE-2017-17091
		CVE-2017-17910
		CVE-2017-17704
		CVE-2017-15654
		CVE-2017-16924
		CVE-2018-1266
		CVE-2018-1108
		CVE-2017-16031
		CVE-2018-11045
		CVE-2018-13280
		CVE-2018-15807
		CVE-2018-16239
		CVE-2018-17888
		CVE-2018-18375
		CVE-2018-18531
		CVE-2018-19983
		CVE-2018-1279
		CVE-2018-17987
		CVE-2018-18602
		CVE-2019-0007
		CVE-2019-8919
		CVE-2018-20025

In [41]:
def fetch_cves_with_cpe_cwe(part, vendor, product, version, cwe_id, others = []):
    # NVD API endpoint for CVE data
    nvd_url = "https://services.nvd.nist.gov/rest/json/cves/2.0"

    # Construct the CPE string for querying
    cpe_string = "cpe:2.3:" + part + ":" + vendor + ":" + product + ":" + version

    if others.__len__() > 7:
      print("Too many optional parameters.")
      return
    # If optional parameters are available
    elif others.__len__() == 0:
      print("No optional parameters.")
    else:
      for other in others:
          cpe_name += ":" + other

    #cpe_string = f"?cpeName=cpe:2.3:a:{asset_name}:{asset_version}"
    cwe_string = f"CWE-{cwe_id}"
    # Query the NVD API for CVEs related to the specified asset
    response = requests.get(f"{nvd_url}?cpeName={cpe_string}&cweID={cwe_string}")
    #response = requests.get(f"{nvd_url}?cpeName={cpe_string}")
    if response.status_code == 200:
        data = response.json()
        #print(data)
        # Initialize a list to store CVE IDs
        tot_res = data['totalResults']
        res_per_page = data['resultsPerPage']
        results_start = data['startIndex']

        if not tot_res:
            print("No CVEs found for this asset.")
            return
        print(f"Total results {tot_res}\nTotal result per page {res_per_page}")
        cve_list = []
        vulnerabilities = data['vulnerabilities']

        # Extract CVE IDs from the vulnerabilities section
        for vulnerability in vulnerabilities:
          cve = vulnerability['cve']
          cve_id = cve['id']
          cve_list.append(cve_id)

        # Print the list of CVE IDs
        print("List of CVEs:")
        for cve in cve_list:
            print(cve)

    else:
        print(f"Failed to fetch data: {response.status_code}")

In [45]:
# Input asset details
cwe_id = input("Enter the CWE ID (e.g., 'CWE-123'): CWE-")
product = input("Enter the product name: ")
version = input("Enter the version: ")
vendor = input("Enter the vendor name: ")
part = input("Enter the part (e.g., 'a'): ")

# If part is valid
if part in parts:
  fetch_cves_with_cpe_cwe(part, vendor, product, version, cwe_id)

else:
  print("Invalid part.")



Enter the CWE ID (e.g., 'CWE-123'): CWE-20
Enter the product name: windows10
Enter the version: 1607
Enter the vendor name: microsoft
Enter the part (e.g., 'a'): o
No optional parameters.
No CVEs found for this asset.
